<a href="https://colab.research.google.com/github/rbhattar07/Python-Projects--Data-Analysis/blob/main/Twitter_sentiment_Analysis_v0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial necessities
- Mounting with Google drive
- Installing kaggle and uploading API token

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
from google.colab import files

files.upload() 

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"riddhbhattar","key":"05af002369fff0d5273085388121e8be"}'}

In [3]:
!ls -lha kaggle.json
# To make sure the API kaggle file has been uploaded

-rw-r--r-- 1 root root 68 Apr  7 17:23 kaggle.json


In [5]:
!pip install -q kaggle
# Installing kaggle API client

In [6]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [7]:
!chmod 600 /root/.kaggle/kaggle.json
# Giving the necessary permisions

In [8]:
!pwd

/content


In [9]:
!kaggle datasets list

ref                                                                       title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
------------------------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
datasets/ankanhore545/cost-of-living-index-2022                           Cost of Living Index 2022                          176KB  2022-03-26 04:56:04           2115         30  1.0              
datasets/piterfm/2022-ukraine-russian-war                                 2022 Ukraine Russia War                              2KB  2022-04-07 09:53:48           5910        355  1.0              
datasets/kamilpytlak/personal-key-indicators-of-heart-disease             Personal Key Indicators of Heart Disease             3MB  2022-02-16 10:18:03           8949        195  1.0              
datasets/kuchhb

In [10]:
!kaggle datasets download -d arkhoshghalb/twitter-sentiment-analysis-hatred-speech

  0% 0.00/1.89M [00:00<?, ?B/s]
100% 1.89M/1.89M [00:00<00:00, 170MB/s]


In [11]:
!unzip /content/twitter-sentiment-analysis-hatred-speech.zip

Archive:  /content/twitter-sentiment-analysis-hatred-speech.zip
  inflating: test.csv                
  inflating: train.csv               


- Two files have been found after we unzipped twitter-sentiment-analysis-hatred-speech.zip 
 - test.csv
 - train.csv 

#### Import required libraries

In [13]:
import os
import math
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# Data
- Download the CSV data using pandas.
- Understand the datafile and explore it.
- What is your objective?
- What kind of analysis are you planning to perform? 

In [17]:
# Download the CSV data using pandas.
df = pd.read_csv('/content/test.csv')
df.info()

# About the datafile:
# There are 17197 rows and the entries are staring from 0.
# There are two columns ID and Tweet
# There are non null or N/A values in the data
# The datatypes that we will be dealing with are int64 and object(1) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17197 entries, 0 to 17196
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      17197 non-null  int64 
 1   tweet   17197 non-null  object
dtypes: int64(1), object(1)
memory usage: 268.8+ KB


### Objective:

The main objective is to  detect hate speech in tweets. For the sake of simplicity, we say a tweet contains hate speech if it has a racist or sexist sentiment associated with it and other than that we will be focusing on understadning the space that each and every type of emotion covers in the twitter world.
- We can hope to get insights.

We will be focusing on quantitative and sentimental analysis.
## Basic emotions:
- Happy
- Sad
- Fear
- Disgust
- Anger
- Surprise

# Acknowledgements
Dataset is provided by Analytics Vidhya

In [18]:
df

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."
...,...,...
17192,49155,thought factory: left-right polarisation! #tru...
17193,49156,feeling like a mermaid ð #hairflip #neverre...
17194,49157,#hillary #campaigned today in #ohio((omg)) &am...
17195,49158,"happy, at work conference: right mindset leads..."


In [134]:
# Creating a basic list of happy words which I'll be updating from time to time
happy_words = np.array(['baby','overjoyed','christmas',
                        'halloween','festival','success','successfull', 'birthday',  'god bless you','god bless', 'good person','bliss', 'bless', 'blessing',
                        'advantageous','congratulations', 'gift', 'thank god', 'congrats', 'good luck', 'blessings', 'blessed', 'over the moon', 'in seventh heaven',
                        'favorable', 'opportune', 'opportunity','on top of the world', 'happy', 'ecstatic','joy', 'gratitude', 'pride', 'serenity', 'interest', 'amusement',
                        'amusing', 'amused', 'interesting', 'interested', 'proud', 'hopefully', 'hope', 'hopefull', 'inspiration', 'inspire', 'aww', 'awe', 'awww', 'love',
                        'content', 'cheer', 'cheerful', 'contented', 'good', 'amazing', 'wonderful', 'wow', 'excited', 'joy', 'joyful', 'overjoyed', 'delighted', 'smile',
                        'smiling', 'beautiful', 'benefit', 'beneficial', 'auspicious', 'auspicion', 'luck', 'lucky', 'achieve', 'achievement', 'achieved', 'mermaid', 'glad', 'colorful', 'gain', 'dope'
                        'sexy', 'beautifull', 'beautiful', 'pretty', 'flower', 'flowers', 'saved', 'wow', 'win', 'hurrah!', 'cute doggy', 'cute', 'pregnant', 'marry', 'married', 'lit af', 'lit'])



# I'll be using for loop and run all the words of happy_words/sad_words in the tweets that df contains and create a list for further analysis
happy_tweets = []
for tweet in df.tweet:
  for x in happy_words:
    if x in tweet:
      happy_tweets.append(tweet)

len(happy_tweets)

# Converting happy_tweets list into pandas dataframe and deleting all the duplicates
happy_tweets = pd.DataFrame(happy_tweets, columns = ['happy_tweets']).drop_duplicates()
print(happy_tweets)
print('The total number of happy tweets:', len(happy_tweets))

                                           happy_tweets
0       3rd #bihday to my amazing, hilarious #nephew...
2     #finished#tattoo#inked#ink#loveitâ¤ï¸ #â¤ï¸...
3     #delicious   #food #lovelife #capetown mannaep...
4     good food, good life , #enjoy and   ððð...
8     i ate,i ate and i ate...ðð   #jamaisasth...
...                                                 ...
9519  we love the pretty, happy and fresh you! #teen...
9522  thought factory: left-right polarisation! #tru...
9523  feeling like a mermaid ð #hairflip #neverre...
9524  #hillary #campaigned today in #ohio((omg)) &am...
9526  my   song "so glad" free download!  #shoegaze ...

[5804 rows x 1 columns]
The total number of happy tweets: 5804


In [136]:
# Creating a basic list of sad words which I'll be updating from time to time
sad_words = np.array(['loss', 'sorrow', 'sad', 'sadness', 'suffering', 'pain', 'depressed', 'depressing', 'anxiety', 'stress', 'trauma', 'ptsd', 
                      'therapy', 'disease', 'sick', 'death', 'accident', 'crime', 'criminal', 'heartbreak', 'heartbreaking', 'mourn', 'mourning', 'mournful',
                      'melancholy', 'hearttrending',  'cast down', 'crestfallen', 'dejected', 'depressed', 'despondent', 'disconsolate', 'doleful', 'down',
                      'downcast', 'downhearted', 'down in the mouth', 'droopy', 'forlorn', 'gloomy', 'glum', 'hangdog', 'heartbroken', 'heartsick', 'heartsore',
                      'heavyhearted', 'inconsolable', 'joyless', 'low', 'low-spirited', 'melancholic', 'melancholy', 'miserable', 'mournful', 'saddened', 
                      'sorrowful', 'sorry', 'unhappy', 'woebegone', 'woeful', 'wretched', 'loss', 'lost', 'lose', 'aggrieved', 'distressed', 'troubled', 'uneasy',
                      'unquiet', 'upset', 'worried', 'despairing', 'hopeless', 'sunk', 'disappointed', 'discouraged', 'disheartened', 'dispirited', 'suicidal', 'dolorous',
                      'lachrymose', 'lugubrious', 'plaintive', 'tearful', 'regretful', 'rueful', 'agonized', 'anguished', 'grieving', 'wailing', 'weeping', 'bleak',
                      'cheerless', 'comfortless', 'dark', 'darkening', 'depressing', 'desolate', 'dismal', 'drear', 'dreary', 'elegiac','elegiacal', 'funereal', 'morbid', 
                      'morose', 'murky', 'saturnine', 'somber','sombre', 'sullen', 'death', 'died', 'dead', 'passed away', 'heart attack', 'racist'])

# I'll be using for loop and run all the words of sad_words in the tweets that df contains and create a list for further analysis
sad_tweets = []
for tweets in df.tweet:
  for y in sad_words:
    if y in tweets:
      sad_tweets.append(tweets)

len(sad_tweets)

# Converting happy_tweets list into pandas dataframe and deleting all the duplicates
sad_tweets = pd.DataFrame(sad_tweets, columns = ['sad_tweets']).drop_duplicates()
print(sad_tweets)
print('The total number of sad tweets:', len(sad_tweets))

                                             sad_tweets
0     i'll stand behind this #guncontrolplease   #se...
1     never been this down on myself in my entire li...
2     can #lighttherapy help with #sad or #depressio...
4     lipo-light helped shape her, and it can help s...
5      @user 8000 followers...we are blessed by all ...
...                                                 ...
2471  50 people are dead? who cares? look at me! loo...
2472  new target  #painting #streeta #graffiti #pain...
2473  @user yay! 3 more chances to talk about me and...
2474     @user @user are the most racist pay ever!!!!! 
2475  my   song "so glad" free download!  #shoegaze ...

[1924 rows x 1 columns]
The total number of sad tweets: 1924
